# MNIST Digit Classification using CNNs

In this assignment, a simple CNN model will be implemented and be trained from scratch. 

## Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np
import tensorflow.keras as Keras
from keras.datasets import mnist

## Prepare Dataset

In [38]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = np.array(X_train, np.float32), np.array(X_test, np.float32) 

# Normalization
X_train, X_test = X_train/255.0, X_test/255.0

First lets set the training configurations

In [ ]:
num_classes = 10

# Training parameters
learning_rate = 0.001
training_steps = 200
batch_size = 128
display_step = 10

# Network parameters
conv_filters = [32, 64]
fc_units = 1024

In [ ]:
# Create batch of the images using Tensors
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

## Model

In [32]:
class ConvNet(Model):
  def __init__(self):
    super(ConvNet, self).__init__()
    
    # Conv 1
    self.conv1 = layers.Conv2D(conv_filters[0], kernel_size=5, activation='relu')
    self.maxpool1 = layers.MaxPool2D(2, strides=2)
    # Conv 2
    self.conv2 = layers.Conv2D(conv_filters[1], kernel_size=3, activation='relu')
    self.maxpool2 = layers.MaxPool2D(2, strides=2)

    self.flatten = layers.Flatten()
    self.fc1 = layers.Dense(1024)
    self.dropout = layers.Dropout(rate=0.2)
    self.out = layers.Dense(num_classes)

  def call(self, x, is_training=False):
      x = tf.reshape(x, [-1, 28, 28, 1])
      x = self.conv1(x)
      x = self.maxpool1(x)
      x = self.conv2(x)
      x = self.maxpool2(x)
      x = self.flatten(x)
      x = self.fc1(x)
      x = self.dropout(x, training=is_training)
      x = self.out(x)

      # test
      if not is_training: 
          x = tf.nn.softmax(x)

      return x

## Loss Function & Metrics

In [33]:
def cross_entropy_loss(x, y):
    y = tf.cast(y, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)

    return tf.reduce_mean(loss)


def accuracy(y_pred, y_true):
    correct_predictions = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))

    return tf.reduce_mean(tf.cast(correct_predictions, tf.float32), axis=-1)


## Optimization Process

In [34]:
conv_net = ConvNet()
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

@tf.function
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = conv_net(x, is_training=True)
        loss = cross_entropy_loss(pred, y)

    trainable_variables = conv_net.trainable_variables
    gradients = g.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

## Train

In [35]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)

    if step % display_step == 0 :
        pred = conv_net(batch_x)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)

        print('Step %i, loss: %f, accuracy: %f' % (step, loss, acc))

Step 10, loss: 1.875519, accuracy: 0.742188
Step 20, loss: 1.646640, accuracy: 0.859375
Step 30, loss: 1.612026, accuracy: 0.898438
Step 40, loss: 1.539526, accuracy: 0.976562
Step 50, loss: 1.567354, accuracy: 0.929688
Step 60, loss: 1.524862, accuracy: 0.976562
Step 70, loss: 1.556172, accuracy: 0.929688
Step 80, loss: 1.554147, accuracy: 0.945312
Step 90, loss: 1.511907, accuracy: 0.984375
Step 100, loss: 1.490669, accuracy: 0.992188
Step 110, loss: 1.483569, accuracy: 0.992188
Step 120, loss: 1.508768, accuracy: 0.992188
Step 130, loss: 1.536074, accuracy: 0.937500
Step 140, loss: 1.511279, accuracy: 0.953125
Step 150, loss: 1.499336, accuracy: 0.976562
Step 160, loss: 1.506891, accuracy: 0.976562
Step 170, loss: 1.508114, accuracy: 0.976562
Step 180, loss: 1.481401, accuracy: 0.992188
Step 190, loss: 1.499523, accuracy: 0.976562
Step 200, loss: 1.485011, accuracy: 1.000000


## Test

In [39]:
pred = conv_net(X_test)
print('Test accuracy: {}'.format(accuracy(pred, y_test)))

Test accuracy: 0.9761999845504761
